In [1]:
import numpy as np
import scipy as sp
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import netCDF4
import sys
%matplotlib inline

font = {'weight' : 'bold', 'size' : 14}
matplotlib.rc('font', **font)

# Define function

In [2]:
def get_gridarea(lon1,lon2,lat1,lat2,R):
    """ calculate grid area a function of latitude and longitude on earth'ssurface """
    s = 2*np.pi*(R)**2 * abs(np.sin(np.pi*lat1/180)-np.sin(np.pi*lat2/180)) * abs(lon1-lon2)/360
    return s
    

In [3]:
# set path of input and output
IN_PATH = "D:\\MyData\\icosdata\\carbontracker\\monthly_global\\merge\\"
OUT_PATH = "D:\\MyData\\jupter\\icos\\ctracker_model\\"
# load data
ncdat = netCDF4.Dataset(IN_PATH + 'CT2018_2000_2017_monthly.nc')

# CarbonTracker NetCDF file information 

In [4]:
print(ncdat.variables.keys())

dict_keys(['date', 'date_bnds', 'longitude', 'latitude', 'levels', 'boundary', 'co2', 'pressure', 'gph', 'idate'])


In [5]:
co2 = ncdat.variables['co2'] # (mol/mol)
print(co2)

<class 'netCDF4._netCDF4.Variable'>
float32 co2(date, levels, latitude, longitude)
    standard_name: mole_fraction_of_carbon_dioxide_in_air
    long_name: mole_fraction_of_carbon_dioxide_in_air
    units: mol mol-1
unlimited dimensions: date
current shape = (216, 25, 180, 360)
filling on, default _FillValue of 9.969209968386869e+36 used


In [6]:
pressure = ncdat.variables['pressure'] # (Pa)
print(pressure)

<class 'netCDF4._netCDF4.Variable'>
float32 pressure(date, boundary, latitude, longitude)
    standard_name: air pressure
    long_name: pressure_at_level_boundaries
    units: Pa
unlimited dimensions: date
current shape = (216, 26, 180, 360)
filling on, default _FillValue of 9.969209968386869e+36 used


In [10]:
gph = ncdat.variables['gph'] # (m)
print(gph)

<class 'netCDF4._netCDF4.Variable'>
float32 gph(date, boundary, latitude, longitude)
    standard_name: geopotential_height
    long_name: geopotential_height_at_level_boundaries
    units: m
unlimited dimensions: date
current shape = (216, 26, 180, 360)
filling on, default _FillValue of 9.969209968386869e+36 used


# Calculate C mass

In [11]:
# 4d lon and lat
arr_lat1 = np.tile(np.arange(90,-90,-1),(360,1)).T
arr_lat2 = np.tile(np.arange(89,-91,-1),(360,1)).T
arr_lat1_3d = np.repeat(arr_lat1[np.newaxis, :, :], 25, axis=0)
arr_lat2_3d = np.repeat(arr_lat2[np.newaxis, :, :], 25, axis=0)
arr_lat1_4d = np.repeat(arr_lat1_3d[np.newaxis, :, :], 216, axis=0)
arr_lat2_4d = np.repeat(arr_lat2_3d[np.newaxis, :, :], 216, axis=0)

lon1 = 1
lon2 = 0

# calculate difference along levels 
arr_gphdiff_4d = np.array(abs(gph[:, 0:25, :, :] - gph[:, 1:26, :, :]))
gph = gph[:, 0:25, :, :]

# netcdf variable to array
co2 = np.asarray(co2)

# 4d radiu of each level
Radiu_earth = 6378.1370 # (km) 
arr_Radiu_4d = gph/1000+Radiu_earth

In [13]:
# calculate grid area (km2), following line will take a lot of memory
arr_s_4d = get_gridarea(lon1, lon2, arr_lat1_4d, arr_lat2_4d, arr_Radiu_4d) 

# if the function get_gridarea is too heavy in this case, then using folling lines
# part1 = 2*np.pi*(arr_Radiu_4d)**2 
# part2 = abs(np.sin(np.pi*arr_lat1_4d/180)-np.sin(np.pi*arr_lat2_4d/180)) 
# part3 = abs(arr_lon1_4d-arr_lon2_4d)/360
# arr_s_4d = part1 * part2 * part3

arr_s_4d = np.array(arr_s_4d.round(4))


# calculate difference along levels 
arr_pdiff_4d = np.array(abs(pressure[:, 0:25, :, :] - pressure[:, 1:26, :, :]))

# Calculate air mass,  (N/m2) * m2 * (N/kg) = kg
arr_airmass_4d = arr_pdiff_4d * arr_s_4d * 1e6 / 9.81   

# calculate CO2 and C mass, (mol/mol) * (g/mol) / (g/mol) * kg = kg
arr_co2mass_4d = co2 * 44.009 / 28.9647 * arr_airmass_4d
arr_cmass_4d = co2 * 12.011 / 28.9647 * arr_airmass_4d

arr_co2ppm_4d = co2 * 1e6

In [13]:
# save the NumPy arrays into a native binary format (.npy)
np.save(OUT_PATH+'arr_co2mass_4d.npy', arr_co2mass_4d / 1e12) # kg -> Gt
np.save(OUT_PATH+'arr_cmass_4d.npy', arr_cmass_4d / 1e12) # kg -> Gt
np.save(OUT_PATH+'arr_co2ppm_4d.npy', arr_co2ppm_4d) # ppm
# np.save(OUT_PATH+'arr_Radiu_4d.npy', np.array(arr_Radiu_4d)) # km
np.save(OUT_PATH+'arr_pdiff_4d.npy', arr_pdiff_4d) # Pa
np.save(OUT_PATH+'arr_s_4d.npy', arr_s_4d) # km2
np.save(OUT_PATH+'arr_gphdiff_4d.npy', arr_gphdiff_4d) # m

# delete variables and clean memory
# del arr_co2mass_4d
# del arr_cmass_4d
# del arr_co2ppm_4d
# del arr_Radiu_4d
# del arr_pdiff_4d

In [ ]:
# list all of variable and its size
local_vars = list(locals().items())
for var, obj in local_vars:
    print(var, sys.getsizeof(obj))